In [151]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

# import glob, os
# glob.glob(os.path.join(os.environ["SPARK_HOME"], "conf", "spark*"))
# print(os.environ)

In [152]:
#read csv

testdf = pd.read_csv('datasets/test.csv')
# traindf = pd.read_csv('datasets/train.csv')
alldf = pd.read_csv('datasets/submit.csv')

# print(testdf.shape, traindf.shape)

# print('cleaning dataframes')

# print(np.sum(testdf.isnull()))

testdf[['title', 'author']] = testdf[['title', 'author']].fillna(' ')

testdf = testdf.dropna()
# traindf = traindf.dropna()

# print(testdf.shape, traindf.shape)

ytestdf = testdf.drop(['title', 'author', 'text'], axis=1)

In [153]:
ytestdf = ytestdf.merge(alldf, left_on='id', right_on='id', how='left')[['id', 'label']]
# ytestdf = pd.merge(left = ytestdf, right=alldf, left_on='id', right_on='id', how='left')
ytest = ytestdf.label
print(ytest.shape)

(5193,)


In [154]:
spark = SparkSession.builder.\
    master('local[*]').\
        config('spark.driver.memory', '10g').\
            appName('fakenews').\
                getOrCreate()

print(spark.sparkContext.uiWebUrl)
print(spark.sparkContext.getConf().getAll())
    # master('local[*]').\
    #     config('spark.driver.memory','10g').\
print(spark)

http://d-i89-236-125.student.eduroam.uq.edu.au:4040
[('spark.app.id', 'local-1648610385004'), ('spark.app.name', 'fakenews'), ('spark.sql.warehouse.dir', 'file:/Users/rushi/Documents/GitHub/fake-news-detection/spark-warehouse'), ('spark.driver.memory', '10g'), ('spark.executor.id', 'driver'), ('spark.driver.port', '65156'), ('spark.app.startTime', '1648610384967'), ('spark.driver.host', 'd-i89-236-125.student.eduroam.uq.edu.au'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [155]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import IntegerType, StringType, DoubleType
from pyspark.sql import DataFrame
from pyspark.sql.functions import monotonically_increasing_id

trainschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType()),\
                StructField('label', IntegerType())]) 

testschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType())]) 

trainset = spark.read.csv('datasets/train.csv', header=True, schema=trainschema)
testset = spark.read.csv('datasets/test.csv', header=True, schema=testschema)
allset = spark.read.csv('datasets/submit.csv', header=True, inferSchema=True)


trainset = trainset.fillna(' ', subset=['title', 'author']).dropna(how='any')
testset = testset.fillna(' ', subset=['title', 'author']).dropna(how='any')

print(trainset.count(), testset.count())

# trainset.printSchema()
# testset.printSchema()

12256 5194


In [156]:
# testset.show(10)

testset = testset.join(allset, on='id', how='left')

In [157]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer, StopWordsRemover, IDF
from pyspark.sql.functions import regexp_replace, udf, col, size

#remove punctuation and remove text with less than 10 words
def lower_clean_str(x):
    punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890'
    lowercased_str = x.lower()
    for ch in punc:
        lowercased_str = lowercased_str.replace(ch,'')
    return lowercased_str

udfclean = udf(lower_clean_str, StringType())

trainset = trainset.withColumn('cleaned', udfclean(trainset.text)).dropna()
testset = testset.withColumn('cleaned', udfclean(testset.text)).dropna()

tokenizer = Tokenizer(inputCol='cleaned', outputCol='words')

trainset = tokenizer.transform(trainset)
testset = tokenizer.transform(testset)

trainset = trainset.where(size(col('words')) > 10)
testset = testset.where(size(col('words')) > 10)

testsetdf = testset.toPandas()

# regex = '[,\\-]'

# trainset = trainset.withColumn('text', regexp_replace(trainset.text, regex, ' '))
# testset = testset.withColumn('text', regexp_replace(testset.text, regex, ' '))

#tonkenize text

# tokenizer = Tokenizer(inputCol='cleaned', outputCol='words')
# wordsdata = tokenizer.transform(trainset)

stopwr = StopWordsRemover(inputCol='words', outputCol='filtered')

hashing = HashingTF(inputCol='filtered', outputCol='hashed', numFeatures=700)

hashingIDF = IDF(inputCol='hashed', outputCol='features')

#create a model
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=2)

In [158]:
testsetdf.head()

,id,title,author,text,label,cleaned,words
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0,palo alto calif — after years of scorning t...,"[palo, alto, calif, , —, , , after, years, of,..."
1,20801,Russian warships ready to strike terrorists ne...,nan,"""Russian warships ready to strike terrorists n...",1,russian warships ready to strike terrorists ne...,"[russian, warships, ready, to, strike, terrori..."
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0,videos nodapl native american leaders vow to s...,"[videos, nodapl, native, american, leaders, vo..."
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",1,if at first you don’t succeed try a different ...,"[if, at, first, you, don’t, succeed, try, a, d..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1,mins ago views comments likes for the firs...,"[, mins, ago, , views, , comments, , likes, fo..."


In [159]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

pipeline = Pipeline(stages=[stopwr, hashing, hashingIDF, gbt])

pipeline = pipeline.fit(trainset.select(col('words'), col('label').cast('int')))


predictions = pipeline.transform(testset.select(col('words'))) 

In [160]:
from sklearn.metrics import accuracy_score, classification_report

Yspark_pred = predictions.toPandas()
Yspark_pred = Yspark_pred['prediction']

ytestdf2 = testset.select('label').toPandas()

print(Yspark_pred.shape, ytestdf2.shape)

FK_accuracy = accuracy_score(ytestdf2, Yspark_pred) * 100
FK_classification_report = classification_report(ytestdf2, Yspark_pred)

print('The accuracy of GBT Classifier to predict fake news is {:.2f}%'.format(FK_accuracy))
print('Classification Report: \n', FK_classification_report)

(4280,) (4280, 1)
The accuracy of GBT Classifier to predict fake news is 63.88%
Classification Report: 
               precision    recall  f1-score   support

           0       0.59      0.79      0.67      2020
           1       0.73      0.50      0.59      2260

    accuracy                           0.64      4280
   macro avg       0.66      0.65      0.63      4280
weighted avg       0.66      0.64      0.63      4280



In [161]:
spark.stop()